원-핫 인코딩

In [ ]:
#넘파이를 이용한 원-핫 인코딩
import numpy as np

samples = ['The cat sat on the mat.', 'The dog ate my homework.']

token_index = {}
for sample in samples:
  for word in sample.split():
    if word not in token_index:
      token_index[word] = len(token_index) + 1 #index 0은 사용하지 않음

  max_length = 10 #각 샘플에서 max_length까지 단어만 사용
  results = np.zeros((len(samples), max_length, max(token_index.values()) + 1)) #3차원(2,10,11)

for i, sample in enumerate(samples):
  for j, word in list(enumerate(sample.split()))[:max_length]:
    index = token_index.get(word)
    results[i,j,index] = 1.

In [ ]:
token_index

{'The': 1,
 'cat': 2,
 'sat': 3,
 'on': 4,
 'the': 5,
 'mat.': 6,
 'dog': 7,
 'ate': 8,
 'my': 9,
 'homework.': 10}

In [ ]:
results

array([[[0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],

       [[0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0

In [ ]:
#케라스를 이용한 원-핫 인코딩
from keras.preprocessing.text import Tokenizer

samples = ['The cat sat on the mat.', 'The dog ate my homework.']

tokenizer = Tokenizer(num_words=1000) #가장 빈도가 높은 1,000개 단어만 선택하는 Tokenizer 객체 생성

tokenizer.fit_on_texts(samples) #단어 인덱스 구축

sequences = tokenizer.texts_to_sequences(samples) #문자열을 정수 인덱스의 리스트로 변환

results = tokenizer.texts_to_matrix(samples,mode='binary')

word_index = tokenizer.word_index

In [ ]:
word_index

{'the': 1,
 'cat': 2,
 'sat': 3,
 'on': 4,
 'mat': 5,
 'dog': 6,
 'ate': 7,
 'my': 8,
 'homework': 9}

In [ ]:
sequences

[[1, 2, 3, 4, 1, 5], [1, 6, 7, 8, 9]]

In [ ]:
results

array([[0., 1., 1., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]])

In [13]:
from tensorflow.keras.datasets import imdb

max_features = 10000 #특성으로 사용할 단어의 수
max_length = 20 #사용할 텍스트의 길이

(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=max_features)

In [16]:
from tensorflow.keras import preprocessing

X_train = preprocessing.sequence.pad_sequences(X_train, max_length) #리스트를 (samples, maxlen)크기의 2D 정수 텐서로 변환
X_test = preprocessing.sequence.pad_sequences(X_test, max_length)

In [3]:
X_train.shape

(25000, 20)

In [4]:
y_train.shape

(25000,)

In [5]:
X_test.shape

(25000, 20)

In [6]:
y_test.shape

(25000,)

In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Embedding

model = Sequential()
model.add(Embedding(1000,8,input_length=max_length)) #8은 embedding dimensionality
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics='acc')
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 8)             8000      
                                                                 
 flatten (Flatten)           (None, 160)               0         
                                                                 
 dense (Dense)               (None, 1)                 161       
                                                                 
Total params: 8,161
Trainable params: 8,161
Non-trainable params: 0
_________________________________________________________________


In [8]:
history = model.fit(X_train, y_train,
                    epochs=10,
                    batch_size=32,
                    validation_split=0.2)

Epoch 1/10
625/625 [==============================] - 6s 3ms/step - loss: 0.6672 - acc: 0.6223 - val_loss: 0.6161 - val_acc: 0.6904
Epoch 2/10
625/625 [==============================] - 2s 3ms/step - loss: 0.5567 - acc: 0.7254 - val_loss: 0.5436 - val_acc: 0.7184
Epoch 3/10
625/625 [==============================] - 2s 3ms/step - loss: 0.5058 - acc: 0.7512 - val_loss: 0.5298 - val_acc: 0.7248
Epoch 4/10
625/625 [==============================] - 2s 3ms/step - loss: 0.4890 - acc: 0.7608 - val_loss: 0.5276 - val_acc: 0.7324
Epoch 5/10
625/625 [==============================] - 2s 3ms/step - loss: 0.4796 - acc: 0.7656 - val_loss: 0.5280 - val_acc: 0.7310
Epoch 6/10
625/625 [==============================] - 2s 3ms/step - loss: 0.4722 - acc: 0.7710 - val_loss: 0.5295 - val_acc: 0.7352
Epoch 7/10
625/625 [==============================] - 2s 3ms/step - loss: 0.4647 - acc: 0.7753 - val_loss: 0.5312 - val_acc: 0.7346
Epoch 8/10
625/625 [==============================] - 2s 3ms/step - loss: 0.

In [11]:
model.evaluate(X_test, y_test)

782/782 [==============================] - 1s 2ms/step - loss: 0.5314 - acc: 0.7309


[0.5314226150512695, 0.7309200167655945]